In [2]:
pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import os
from pathlib import Path
import datetime
import argparse
import traceback
import hashlib

def process_file(input_file_path):
    try:
        # base_input_dir = "graph_data/nodes"
        # input_file = base_input_dir + "/Databases.csv"

        df = pd.read_csv(input_file_path, sep=",", header=0, escapechar="\\", quotechar="`")
        print(input_file_path +" Read - The shape of dataframe: "+ str(df.shape).replace(",", " rows, with number of columns:"))
        print(df.loc[:, ['Entity_ID:ID','HashKey']])
        deduped_df = df.drop_duplicates(subset=['HashKey'], keep='last').loc[df["Entity_Name"] != "Entity_Name" ]
        print(input_file_path + " Deduped - The shape of dataframe: "+ str(deduped_df.shape).replace(",", " rows, with number of columns:"))
        print (deduped_df)
        
        os.remove(input_file_path)
        print (input_file_path + " Deleted - The file has been removed")

        from pathlib import Path  
        filepath = Path(input_file_path)  
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        deduped_df.to_csv(filepath, index=False, quotechar='`')
        print (input_file_path + " Recreated - The file has been recreated using deduped data")
        options = ['Entity_Name'] 
    
        # selecting rows based on condition 
        rslt_df = deduped_df[deduped_df['Entity_Name'].isin(options)] 
            
        print('\nResult dataframe :\n',
            rslt_df)
    except BaseException as e:
        print (input_file_path + "Failed - something went wrong in process_file for file "+ str(e))

        # traceback.print_exc
        
def profile_adept_from_folder(base_input_dir):
    """
    profile_adept_results from a folder
    """
    #os.system('pwd')
    #os.system('ls -l')
    now = datetime.datetime.now()
    print (now)
    try:
        ddl_files = Path(base_input_dir).glob('*.csv')
        dfs = list()
        for f in ddl_files:
            print(str(f) + " found...")
            if 'Macros.csv' not in str(f):
                print("Macro file found, skipping file from processing"+str(f))
            # dedupe each csv file found
            process_file(str(f))
    except Exception as e:
        print ("something went wrong in profile_adept_results during profile for a folder "+ str(e))
        raise
    print (now)


profile_adept_from_folder("/Users/Shared/ADEPT/out/DBT_TO_ADEPT/graph_data/nodes")
# process_file('/Users/Shared/ADEPT/out/DBT_TO_ADEPT/graph_data/nodes/Macros.csv')

2023-08-21 19:06:36.156460
2023-08-21 19:06:36.156460


In [ ]:
if __name__ == "__main__":
    """
    Main function: driver for application upload_harvester_output.py
    """
    parser = argparse.ArgumentParser(
                    prog='ADEPT Graph Data Deduper',
                    description='''This program is used to test uploading to the ADEPT Console. This script will upload the output files generated by the harvester job.
Refer: https://bitbucket.org/eoncollective-code-repository/adept-ui-console/src/master/docs/integration-notes/index.md
for how to create nodes and relationships graph data with csv files.''',
                    epilog='ADEPT utilities')
    parser.add_argument('-i','--input',
                    default='/Users/Shared/ADEPT/out/DBT_TO_ADEPT',
                    required=True,
                    help='Provide input location with .csv using a comma as delimiter (default)')
    parser.add_argument('-t','--type',
                        default='folder',
                        choices=['file', 'folder'],
                        required=True,
                        help='Choice to process file of folder for dedupe')
    args = parser.parse_args()
    profile_adept_from_folder(args.input)

# Process One File with Graph Data for documentation in ADEPT

In [32]:
input_file = "/Users/Shared/ADEPT/out/100/graph_data/nodes/Users.csv"
print(input_file.split("/")[-1].split(".")[1])

csv


In [36]:
from datetime import datetime

output_dir="/Users/james.kimani/Development/repositories/data_mart_dumper/src/profiler"
job_name="output"

input_file = "/Users/Shared/ADEPT/out/100/graph_data/nodes/Users.csv"

file_name =  Path(input_file).name
file_extension = file_name.split(".")[1]

df = pd.read_csv(input_file, sep=",", header=0, escapechar="\\", quotechar="`")

# check if graph data directory for nodes exists, if not create it
gd_nodes_directory=output_dir+'/'+job_name+'/graph_data/nodes'
gd_nodes_directory_isExist = os.path.exists(gd_nodes_directory)
if not gd_nodes_directory_isExist:
    # Create a new directory because it does not exist for /graph_data/nodes
    os.makedirs(gd_nodes_directory+"/")
    print("The new directory is created! "+ gd_nodes_directory)

# check if graph data directory for nodes exists, if not create it
gd_relationships_directory=output_dir+'/'+job_name+'/graph_data/relationships'
gd_relationships_directory_isExist = os.path.exists(gd_relationships_directory)
if not gd_relationships_directory_isExist:
    # Create a new directory because it does not exist for /graph_data/relationships
    os.makedirs(gd_relationships_directory+"/")
    print("The new directory is created! "+ gd_relationships_directory)

is_NodeFile_Existing = os.path.exists("output/graph_data/nodes/file.csv") 
if is_NodeFile_Existing == False:
        BaseNodeFile = open("output/graph_data/nodes/file.csv", "a")
        # Schema => Entity_ID:ID,:Label,HashKey,name, Technical Data Type,
        file_column_header_line = ["Entity_ID:ID,:Label,HashKey,name,Path,Extension,Size,Last_Modified_Time,Creation_Time\n"]
        BaseNodeFile.writelines(file_column_header_line)
        file_column_line = ["%s,%s,%s,%s,%s,%s,%s bytes,%s,%s\n"%(hashlib.md5((input_file).encode()).hexdigest(),"File",hashlib.md5((input_file).encode()).hexdigest(),file_name,input_file,file_extension,os.path.getsize(input_file),os.path.getctime(input_file),os.path.getmtime(input_file))]
        BaseNodeFile.writelines(file_column_line)
        BaseNodeFile.close()
else:
    BaseNodeFile = open("output/graph_data/nodes/file.csv", "a")
    # Schema => Entity_ID:ID,:Label,HashKey,name,Path,Extension,Size,Creation_Time,Last_Modified_Time
    file_column_line = ["%s,%s,%s,%s,%s,%s,%s bytes,%s,%s\n"%(hashlib.md5((input_file).encode()).hexdigest(),"File",hashlib.md5((input_file).encode()).hexdigest(),file_name,input_file,file_extension,os.path.getsize(input_file),os.path.getctime(input_file),os.path.getmtime(input_file))]
    BaseNodeFile.writelines(file_column_line)
    BaseNodeFile.close()
          

is_NodeFile_Existing = os.path.exists("output/graph_data/nodes/file_columns.csv") 
if is_NodeFile_Existing == False:
    BaseNodeFile = open("output/graph_data/nodes/file_columns.csv", "a")
    # Schema => Entity_ID:ID,:Label,HashKey,name, Technical Data Type,
    file_column_line = ["Entity_ID:ID,:Label,HashKey,name, Technical Data Type\n"]
    BaseNodeFile.writelines(file_column_line)
    BaseNodeFile.close()       

is_RelationshipFile_Existing = os.path.exists("output/graph_data/relationships/relationships.csv")
if is_RelationshipFile_Existing == False:
    BaseRelationshipFile = open("output/graph_data/relationships/relationships.csv", "a")
    # Schema => Entity_ID:ID,:Label,HashKey,name, Technical Data Type,
    file_column_relationship_line = [":START_ID,:END_ID,:TYPE,timestamp,Version\n"]
    BaseRelationshipFile.writelines(file_column_relationship_line)
    BaseRelationshipFile.close()     

for colname, coltype in df.infer_objects().dtypes.items():       
    print(colname, coltype)
    NodeFileColumn = open("output/graph_data/nodes/file_columns.csv", "a")
    # Schema => Entity_ID:ID,:Label,HashKey,name, Technical Data Type,
    Entity_ID = hashlib.md5((input_file+"~"+str(colname)).encode()).hexdigest()
    Label = "Column"
    HashKey = Entity_ID
    name = str(colname)
    Technical_Data_Type = str(coltype)
    Source = filename
    file_column_line = [Entity_ID + "," + Label + "," + HashKey + "," + name + "," + Technical_Data_Type + "," + Source + "\n"]
    NodeFileColumn.writelines(file_column_line)
    NodeFileColumn.close()
          
    RelationshipFile = open("output/graph_data/relationships/relationships.csv", "a")
    # Schema => :START_ID,:END_ID,:TYPE,timestamp,Version
    START_ID = hashlib.md5((input_file).encode()).hexdigest()
    END_ID = hashlib.md5((input_file+"~"+str(colname)).encode()).hexdigest()
    TYPE = "contains_columns"
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d%H%M%S")
    Version = "1"
    file_column_relationship_line = [START_ID + "," + END_ID + "," + TYPE + "," + timestamp + "," + Version + "\n"]
    RelationshipFile.writelines(file_column_relationship_line)
    RelationshipFile.close()


FileNotFoundError: [Errno 2] No such file or directory: 'output/graph_data/nodes/file.csv'

In [ ]:
# python for_loop_with_range_function.py

str_list = ["Macros.csv", "Models.csv"]

for x in range(len(str_list)):
    print(str_list[x])